In [3]:
%load_ext autoreload
%autoreload 2

In [8]:
pip install --upgrade tqdm

  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
from pyldpc import make_ldpc, encode, decode, get_message

n = 20
d_v = 4
d_c = 5
snr = 200
H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
k = G.shape[1]
v = np.random.randint(2, size=k)
y = encode(G, v, snr)
d = decode(H, y, snr)
x = get_message(G, d)
assert abs(x - v).sum() == 0

In [ ]:
i=0
while v.shape != (8,):
    i+=1
    H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
    v = np.random.randint(2, size=G.shape[1])
    print('aaaa')
print(v.shape)
print(i)

In [78]:
from decoder import bit_flip_dec

In [4]:
decoded_x = bit_flip_dec(H,y)

100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 21708.75it/s]


In [6]:
abs(x - decoded_x).sum() #comparison with pyldpc decoder

ValueError: operands could not be broadcast together with shapes (6,) (15,) 

In [8]:
abs(x - v).sum() #comparison with input msg

(15,)

In [90]:
y.shape

(20,)

In [58]:
from abc import ABC
from tqdm import tqdm
import numpy as np
from pyldpc import make_ldpc, encode, decode, get_message

class BaseAlgo(ABC):
    def __init__(self, algo_name , algo_params = {}):
        super(BaseAlgo, self).__init__()
        self.algo_name = algo_name
        self.algo_params = algo_params
        
    
    def decode(self, y_err):
        raise NotImplementedError

class BitFlipAlgo(BaseAlgo):
    def __init__(self, algo_name, algo_params = {}):
        super(BitFlipAlgo, self).__init__(algo_name=algo_name, algo_params=algo_params)
    
    def bit_flip(self, H, y_err):
        metric = self.algo_params["metric"](H ,y_err)
        bit_flip_idx = self.algo_params["selector"](metric, y_err)
        new_y = y_err
        new_y[bit_flip_idx] = (new_y[bit_flip_idx]  + 1) % 2
        return new_y
        
    def decode(self, H, y_err):
        y_list = [y_err]
        for i in tqdm(range(self.algo_params["max_iter"])):
            syndrome = (H @ y_list[i]) % 2 
            if (syndrome == 0).all():
                return y_list
            else:
                y_list.append(self.bit_flip(H, y_list[i]))
        return y_list

# class ExploreComp(BitFlipAlgo):
#     def __init__(self, algo_name, algo_params = {}):
#         super(ExploreComp, self).__init__(algo_name=algo_name, algo_params=algo_params)
        
#     def bit_flip(self, y_err):
#         if np.random.rand() < self.algo_params["p"]:
#             bit_flip_idx = np.random.choice(y_err.shape[0])
#             new_y = y_err
#             new_y[bit_flip_idx] = (new_y[bit_flip_idx]+1) % 2
#             return new_y
#         else:
#             return super().bit_flip(y_err)

class Metric:
    def __init__(self, name):
        self.name = name
        assert self.name in ["sat", "unsat", "unsat_sat"], "metric {} not implemented".format(self.name)
    def  __call__(self, H, y_err):
        syndrome = (H @ y_err).astype(int) % 2

        if self.name in ["sat", "unsat_sat"]:
            satisfied_parity_idx = np.arange(H.shape[0])[syndrome !=0]
            num_satisfied = H[satisfied_parity_idx].sum()
        if self.name in ["unsat", "unsat_sat"]:
            unsatisfied_parity_idx = np.arange(H.shape[0])[syndrome != 0]
            num_unsatisfied = H[unsatisfied_parity_idx].sum()
        if self.name  == "sat":
            return num_satisfied
        elif self.name == "unsat":
            return num_unsatisfied
        else:
            return num_unsatisfied -  num_satisfied
class BitSelector:
    def __init__ (self, name, params= {"lambda" : 1}):
        self.name = name
        self.params = params
        assert self.name in ["greedy", "weighted"], "Bit selection method {} not implemented".format(self.name)

    def __call__(self, metric, y_err):
        if self.name == "greedy":
            return np.argmax(metric)
        else:
            return np.random.choice(np.arange(metric.shape[0]),p=np.exp(self.params["lambda"]* metric))

class ExploreSelector(BitSelector):
    def __init__(self, name,params = {"lambda": 1, "p": 0.2}):
        super(ExploreSelector, self).__init__(name = name, params=params)
    def __call__(self, metric, y_err):
        if np.random.rand() <= self.params["p"]:
            return np.random.choice(np.arange(metric.shape[0]))
        else:
            return super().__call__(metric, y_err)

# metric = 

In [45]:
METRIC_CHOICES = ["sat", "unsat","unsat_sat"]
SELECTOR_CHOICES = ["greedy", "weighted"]
ALGO_CHOICES = []
metric_name = METRIC_CHOICES[0] 
selector_name = SELECTOR_CHOICES[0]
selector = BitSelector(selector_name)
metric = Metric(metric_name)

algo_name = metric_name + "_" + selector_name
algo_params = {"max_iter" : int(1e3), "metric" : metric, "selector":selector}



In [46]:
algo = BitFlipAlgo(algo_name=algo_name, algo_params=algo_params)

In [61]:
class Stats():
    def __init__(self, l1_dist, absolute_correct, is_codeword):
        self.l1_dist = l1_dist
        self.absolute_correct = absolute_correct
        self.is_codeword = is_codeword

In [62]:
import itertools

def benchmark_algo(algorithm, codes_per_case, trials_per_code):
    stats = {}
    n = [10, 100, 1000]
    # rate = [0.01, 0.1, 0.5]
    frac_of_errs = [0.1, 0.25, 0.4]
    cases = list(itertools.product(n, frac_of_errs))
    d_v = 4
    d_c = 5
    snr = 200


    for case in tqdm(cases):
        n, frac_of_errs = case
        
        for code in range(codes_per_case):
            H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
            k = G.shape[1]
            v = np.random.randint(2, size=k)
            y = encode(G, v, snr)
            
            l1_dist = 0
            num_abs_correct = 0
            num_valid_codeword = 0
            
            for trial in range(trials_per_code):
                y = add_error(y, int(frac_of_errs*n))
                
                y_list = algorithm.decode(H, y)
                decode_y = y_list[-1]
                y = y.astype(int)
                decode_y = decode_y.astype(int)
                l1_dist += np.absolute(decode_y - y)
                num_abs_correct += 1 if (decode_y == y).all() else 0
                num_valid_codeword += 1 if (H@decode_y).all() == 0 else 0

            stats[code] = Stats(l1_dist/trials_per_code, num_abs_correct/trials_per_code, num_valid_codeword/trials_per_code)
            
    return stats

In [63]:
stats = benchmark_algo(algo,1,1)

  0%|          | 0/9 [00:00<?, ?it/s]/tmp/ipykernel_68230/2104473358.py:31: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if syndrome == np.zeros(y.shape[0]):
100%|██████████| 9/9 [00:23<00:00,  2.66s/it]


In [64]:
from decoder import add_error

In [70]:
stats[0].absolute_correct

0.0